In [6]:
# import subprocess
# subprocess.run("ollama serve", shell=True)

In [7]:
# import ollama
# #Then everytime you want to chat
# response = ollama.chat(model='llama3.2', messages=[
#   {
#     'role': 'user',
#     'content': 'Why is the sky blue?',
#   },
# ])
# print(response['message']['content'])

In [1]:
import pandas as pd
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

In [2]:
from dotenv import load_dotenv
load_dotenv() # read local .env file
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [3]:
import streamlit as st
# from streamlit_extras.add_vertical_space import add_vertical_space
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os

In [4]:
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)


In [10]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    deployment= 'genai-gpt-35-turbo',
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    azure_endpoint=OPENAI_API_BASE, #"https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    api_key=OPENAI_API_KEY, # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    openai_api_version=OPENAI_API_VERSION, # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)

In [8]:
import time

In [14]:
# # Function to process text data in chunks and add to the FAISS index
# def process_in_chunks(text_data, chunk_size, embeddings, index_name):
#     # Check if the index already exists
#     if os.path.exists(f"{index_name}.faiss") and os.path.exists(f"{index_name}.pkl"):
#         vectorstore = FAISS.load_local(index_name, embeddings)
#         print("Loaded existing FAISS index.")
#     else:
#         vectorstore = None
    
#     for i in range(0, len(text_data), chunk_size):
#         chunk = text_data[i:i + chunk_size]
#         try:
#             print(f"Processing chunk {i // chunk_size + 1}")
#             temp_vectorstore = FAISS.from_texts(chunk, embeddings)
            
#             if vectorstore:
#                 vectorstore.merge_from(temp_vectorstore)
#             else:
#                 vectorstore = temp_vectorstore
            
#             # Save the updated FAISS index after processing each chunk
#             vectorstore.save_local(index_name)
#             print(f"Chunk {i // chunk_size + 1} processed and index saved.")
            
#         except Exception as e:
#             print(f"Error processing chunk {i}: {e}")
        
#         time.sleep(1)  # Sleep to respect the rate limit
    
#     return vectorstore



In [9]:
# Initialize embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

# Load documents
loader = DirectoryLoader("./input-data", glob="**/*.txt")
print("Directory loaded")
documents = loader.load()

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1500,
    chunk_overlap=300,
    add_start_index=True,
)

# Split documents into chunks
texts = text_splitter.split_documents(documents)

Directory loaded


In [13]:
# Create FAISS index
vectorstore = FAISS.from_documents(texts, embeddings,)

print("FAISS index created successfully")

# Save the index locally
vectorstore.save_local("my_faiss_index")

print("FAISS index saved locally")

FAISS index created successfully
FAISS index saved locally


In [14]:
# Load the saved index
loaded_vectorstore = FAISS.load_local("my_faiss_index", embeddings, allow_dangerous_deserialization=True)

# Create retriever
retriever = loaded_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)



In [18]:
load_dotenv() # read local .env file
OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [19]:
llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY,
                        openai_api_type=OPENAI_API_TYPE,
                        #deployment_name = 'genai-gpt-4-32k',
                        #model_name = 'gpt-4-32k')
                        deployment_name = 'genai-gpt-35-turbo',
                        model_name = 'gpt-35-turbo')

C:\Users\750040697\AppData\Local\Temp\ipykernel_1944\3925094818.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,


In [20]:
# # Create prompt template
template = """<bos><start_of_turn>user
You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based only on the following context and provide a meaningful response. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the answer, politely state that you don't have the specific information and suggest contacting the sales department for more details.

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template )

# # Modify the rag_chain to return the result instead of streaming
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
rag_chain.invoke("RfP affiliation change request")

'All affiliation change requests must be accompanied by documentation (such as an email) from the account requesting such. Please include account number, group to be affiliated with, and quarter to begin new affiliation. Alternatively, the account can send the request to RA VISUS SalesInquir@its.jnj.com directly. Ensure that they include SAP, date of affiliation change, and group to be affiliated with. An account must join, or have joined, the requested RfP group on or before the last day of the first month of the quarter to qualify for a rebate.'

In [15]:
# Now you can use the retriever in your RAG pipeline
# # For example:
# query = "RfP affiliation change request"
# relevant_documents = retriever.get_relevant_documents(query)

# print(f"Retrieved {len(relevant_documents)} relevant documents")
# for doc in relevant_documents:
#     print(doc.page_content[:100] + "...")

C:\Users\750040697\AppData\Local\Temp\ipykernel_1944\4169773792.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_documents = retriever.get_relevant_documents(query)


Retrieved 5 relevant documents
RfP Alignments and Affiliations

Account is not found on the RfP Tracker

An account will not appear...
VISUS

SalesInquir@its.jnj.com directly. Ensure that they include SAP, date of affiliation change, a...
You may view quarterly RFP history in the Preferred RFP Tracker in the ‘Rebate History’ section. ACU...
If it is myAPP, Alliance or LF ; Refer to the CX team

Option to Send Inquiry

Name or address on pa...
Rebate Percentage Errors in the Rfp Tracker This is only a reporting issue in the Rfp tracker and di...
